# Module 5: Hierarchical Generators


## What you will learn
* How to write layout generators that instantiate other generators
* How to write schematic generators that instantiate other generators
* Implement an amplifier chain example

## AmpChain Example
<img src="bootcamp_pics/5_hierarchical_generator/hierachical_generator_1.PNG" />
* AmpCS and AmpSF
* Connect bottom VSS together
    * Ignore AmpSF top VSS
* Connect vmid with vertical track in middle
* Connect VDD to top-most M5 track
* Export other ports in-place

## AmpChain Class
```python
class AmpChain(TemplateBase):
    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(AmpChain, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        return dict(
            cs_params='common source amplifier parameters.',
            sf_params='source follower parameters.',
            show_pins='True to draw pin geometries.',
        )
```
* Subclass TemplateBase (the most basic layout class)
* Layout parameters don't have to be scalars; they can be complex python data structures (dictionaries, lists, etc.)

## Creating Layout Master
```python
# create layout masters for subcells we will add later
cs_master = self.new_template(params=cs_params, temp_cls=AmpCS)
# TODO: create sf_master
sf_master = None'
```
* self.new_template() creates a new layout master which is an instance of the Python class temp_cls
    * Master: a layout cellview
    * Instance: and instance of a layout cellview in the current layout, shifted/rotated
        

## Create Layout Instance
```python
# add subcell instances
cs_inst = self.add_instance(cs_master, 'XCS')
# add source follower to the right of common source
x0 = cs_inst.bound_box.right_unit
sf_inst = self.add_instance(sf_master, 'XSF', loc=(x0, 0), unit_mode=True)
```
* self.add_instance() creates an instance of a layout master
    * Default location at (0, 0), no rotation
* The `bound_box` attribute returns the bounding box of the instance
    * Used here to abut AmpSF to the right of AmpCS

## Get Instance Ports
```python
# get subcell ports as WireArrays so we can connect them
vmid0 = cs_inst.get_all_port_pins('vout')[0]
vmid1 = sf_inst.get_all_port_pins('vin')[0]
vdd0 = cs_inst.get_all_port_pins('VDD')[0]
vdd1 = sf_inst.get_all_port_pins('VDD')[0]
```
* get_all_port_pins() returns a list of all pins (as WireArrays) on the given port on the given layer
    * Port: a logical net. A port can contain many pins on many layers
    * Pin: a physical rectangle
* Here we know there's only one pin, so we just get the first element of the list

## Routing Grid Object
```python
# get vertical VDD TrackIDs
vdd0_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd0.middle))
vdd1_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd1.middle))
```
* self.grid is a RoutingGrid object, which contains many useful functions related to the routing grid
    * See documentation for more information
* coord_to_nearest_track() returns the track index on the given layer closest to the given coordinate
    * Used here to find vertical tracks to connect VDDs to

## Re-export Pins on Instances
```python
 # re-export pins on subcells.
self.reexport(cs_inst.get_port('vin'), show=show_pins)
self.reexport(cs_inst.get_port('vbias'), net_name='vb1', show=show_pins)
# TODO: reexport vout and vbias of source follower
# TODO: vbias should be renamed to vb2
```
* self.reexport() is a convenience function to re-export a port on an instance as a port of the current layout
* can rename the port by using net_name parameter

## Layout Exercises
* Create layout master for AmpSF
* Using RoutingGrid, connect vmid wires together using the vertical track between the two blocks
    * Hint: variable x0 is the X coordinate of the boundary between the two blocks
* Re-export vout and vbias of source follower. Rename vbias to vb2
* See AmpChainSoln if you're stuck

In [ ]:
import sys
sys.path.append('BAG_XBase_demo/demo_scripts')
sys.path.append('BAG_XBase_demo/xbase_demo/demo_layout')
import bootcamp_demo as d
#import core
from __future__ import (absolute_import, division,
                        print_function, unicode_literals)
# noinspection PyUnresolvedReferences,PyCompatibility
from builtins import *

from bag.layout.routing import TrackID
from bag.layout.template import TemplateBase

from abs_templates_ec.analog_core import AnalogBase

In [ ]:
class AmpChain_new(TemplateBase):
    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(AmpChain, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        return dict(
            cs_params='common source amplifier parameters.',
            sf_params='source follower parameters.',
            show_pins='True to draw pin geometries.',
        )

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        # make copies of given dictionaries to avoid modifying external data.
        cs_params = self.params['cs_params'].copy()
        sf_params = self.params['sf_params'].copy()
        show_pins = self.params['show_pins']

        # disable pins in subcells
        cs_params['show_pins'] = False
        sf_params['show_pins'] = False

        # create layout masters for subcells we will add later
        cs_master = self.new_template(params=cs_params, temp_cls=AmpCS)
        # TODO: create sf_master
        sf_master = None

        if sf_master is None:
            return

        # add subcell instances
        cs_inst = self.add_instance(cs_master, 'XCS')
        # add source follower to the right of common source
        x0 = cs_inst.bound_box.right_unit
        sf_inst = self.add_instance(sf_master, 'XSF', loc=(x0, 0), unit_mode=True)

        # get VSS wires from AmpCS/AmpSF
        cs_vss_warr = cs_inst.get_all_port_pins('VSS')[0]
        sf_vss_warrs = sf_inst.get_all_port_pins('VSS')
        # only connect bottom VSS wire of source follower
        if sf_vss_warrs[0].track_id.base_index < sf_vss_warrs[1].track_id.base_index:
            sf_vss_warr = sf_vss_warrs[0]
        else:
            sf_vss_warr = sf_vss_warrs[1]

        # connect VSS of the two blocks together
        vss = self.connect_wires([cs_vss_warr, sf_vss_warr])[0]

        # get layer IDs from VSS wire
        hm_layer = vss.layer_id
        vm_layer = hm_layer + 1
        top_layer = vm_layer + 1

        # calculate template size
        tot_box = cs_inst.bound_box.merge(sf_inst.bound_box)
        self.set_size_from_bound_box(top_layer, tot_box, round_up=True)

        # get subcell ports as WireArrays so we can connect them
        vmid0 = cs_inst.get_all_port_pins('vout')[0]
        vmid1 = sf_inst.get_all_port_pins('vin')[0]
        vdd0 = cs_inst.get_all_port_pins('VDD')[0]
        vdd1 = sf_inst.get_all_port_pins('VDD')[0]

        # get vertical VDD TrackIDs
        vdd0_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd0.middle))
        vdd1_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd1.middle))

        # connect VDD of each block to vertical M5
        vdd0 = self.connect_to_tracks(vdd0, vdd0_tid)
        vdd1 = self.connect_to_tracks(vdd1, vdd1_tid)
        # connect M5 VDD to top M6 horizontal track
        vdd_tidx = self.grid.get_num_tracks(self.size, top_layer) - 1
        vdd_tid = TrackID(top_layer, vdd_tidx)
        vdd = self.connect_to_tracks([vdd0, vdd1], vdd_tid)

        # TODO: connect vmid0 and vmid1 to vertical track in the middle of two templates
        # hint: use x0
        vmid = None

        if vmid is None:
            return

        # add pins on wires
        self.add_pin('vmid', vmid, show=show_pins)
        self.add_pin('VDD', vdd, show=show_pins)
        self.add_pin('VSS', vss, show=show_pins)
        # re-export pins on subcells.
        self.reexport(cs_inst.get_port('vin'), show=show_pins)
        self.reexport(cs_inst.get_port('vbias'), net_name='vb1', show=show_pins)
        # TODO: reexport vout and vbias of source follower
        # TODO: vbias should be renamed to vb2

        # compute schematic parameters.
        self._sch_params = dict(
            cs_params=cs_master.sch_params,
            sf_params=sf_master.sch_params,
        )

## Testing Layout
* Run the below cell to generate the AmpChain layout
* Cellviews should be generated in DEMO_AMP_CHAIN library

In [ ]:
spec_fname = 'demo_specs/demo.yaml'

# load specifications from file
top_specs = d.read_yaml(spec_fname)

# create BagProject object
local_dict = locals()
if 'bprj' in local_dict:
    print('using existing BagProject')
    bprj = local_dict['bprj']
else:
    print('creating BagProject')
    bprj = d.BagProject()
    
d.gen_layout(bprj, top_specs, 'amp_sf', AmpChain_new)

## AmpChain Schematic
<img src="bootcamp_pics/5_hierarchical_generator/hierachical_generator_2.PNG" />
* Hierarchical schematic is siple; just instantiate the schematic template
    * NOT the generated schematic
* Exercise: instantiate amp_sf, name it XSF, connect it, then use import_design_library() to import schematic to Python

## AmpChain Layout Solution
```python
class AmpChainSoln(TemplateBase):
    def __init__(self, temp_db, lib_name, params, used_names, **kwargs):
        super(AmpChainSoln, self).__init__(temp_db, lib_name, params, used_names, **kwargs)
        self._sch_params = None

    @property
    def sch_params(self):
        return self._sch_params

    @classmethod
    def get_params_info(cls):
        return dict(
            cs_params='common source amplifier parameters.',
            sf_params='source follower parameters.',
            show_pins='True to draw pin geometries.',
        )

    def draw_layout(self):
        """Draw the layout of a transistor for characterization.
        """

        cs_params = self.params['cs_params'].copy()
        sf_params = self.params['sf_params'].copy()
        show_pins = self.params['show_pins']

        cs_params['show_pins'] = False
        sf_params['show_pins'] = False

        # create layout masters for subcells we will add later
        cs_master = self.new_template(params=cs_params, temp_cls=AmpCS)
        sf_master = self.new_template(params=sf_params, temp_cls=AmpSFSoln)

        # add subcell instances
        cs_inst = self.add_instance(cs_master, 'XCS')
        # add source follower to the right of common source
        x0 = cs_inst.bound_box.right_unit
        sf_inst = self.add_instance(sf_master, 'XSF', loc=(x0, 0), unit_mode=True)

        # get VSS wires from AmpCS/AmpSF
        cs_vss_warr = cs_inst.get_all_port_pins('VSS')[0]
        sf_vss_warrs = sf_inst.get_all_port_pins('VSS')
        # only connect bottom VSS wire of source follower
        if sf_vss_warrs[0].track_id.base_index < sf_vss_warrs[1].track_id.base_index:
            sf_vss_warr = sf_vss_warrs[0]
        else:
            sf_vss_warr = sf_vss_warrs[1]

        # connect VSS of the two blocks together
        vss = self.connect_wires([cs_vss_warr, sf_vss_warr])[0]

        # get layer IDs from VSS wire
        hm_layer = vss.layer_id
        vm_layer = hm_layer + 1
        top_layer = vm_layer + 1

        # calculate template size
        tot_box = cs_inst.bound_box.merge(sf_inst.bound_box)
        self.set_size_from_bound_box(top_layer, tot_box, round_up=True)

        # get subcell ports as WireArrays so we can connect them
        vmid0 = cs_inst.get_all_port_pins('vout')[0]
        vmid1 = sf_inst.get_all_port_pins('vin')[0]
        vdd0 = cs_inst.get_all_port_pins('VDD')[0]
        vdd1 = sf_inst.get_all_port_pins('VDD')[0]

        # get vertical VDD TrackIDs
        vdd0_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd0.middle))
        vdd1_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, vdd1.middle))

        # connect VDD of each block to vertical M5
        vdd0 = self.connect_to_tracks(vdd0, vdd0_tid)
        vdd1 = self.connect_to_tracks(vdd1, vdd1_tid)
        # connect M5 VDD to top M6 horizontal track
        vdd_tidx = self.grid.get_num_tracks(self.size, top_layer) - 1
        vdd_tid = TrackID(top_layer, vdd_tidx)
        vdd = self.connect_to_tracks([vdd0, vdd1], vdd_tid)

        # connect vmid using vertical track in the middle of the two templates
        mid_tid = TrackID(vm_layer, self.grid.coord_to_nearest_track(vm_layer, x0, unit_mode=True))
        vmid = self.connect_to_tracks([vmid0, vmid1], mid_tid)

        # add pins on wires
        self.add_pin('vmid', vmid, show=show_pins)
        self.add_pin('VDD', vdd, show=show_pins)
        self.add_pin('VSS', vss, show=show_pins)
        # re-export pins on subcells.
        self.reexport(cs_inst.get_port('vin'), show=show_pins)
        self.reexport(cs_inst.get_port('vbias'), net_name='vb1', show=show_pins)
        self.reexport(sf_inst.get_port('vout'), show=show_pins)
        self.reexport(sf_inst.get_port('vbias'), net_name='vb2', show=show_pins)

        # compute schematic parameters.
        self._sch_params = dict(
            cs_params=cs_master.sch_params,
            sf_params=sf_master.sch_params,
        )
```